In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
data = pd.read_csv("D:\Downloads\mnist_train.csv")
data.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
data = np.array(data)
m, n = data.shape    
# m is the amount of rows, n is the amount of features
np.random.shuffle(data) 
# shuffle before splitting into dev and training sets
# transpose so that each column beacomes an example
data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
# normalise
X_dev = X_dev / 255.


data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [4]:
# initialising parameters
def initialise_parameters():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_propagation(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_propagation(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [5]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = initialise_parameters()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_propagation(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_propagation(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [6]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

Iteration:  0
[0 0 0 ... 0 5 0] [0 4 3 ... 8 0 0]
0.0815593220338983
Iteration:  10
[0 2 4 ... 5 6 0] [0 4 3 ... 8 0 0]
0.13428813559322034
Iteration:  20
[0 2 4 ... 5 6 0] [0 4 3 ... 8 0 0]
0.220135593220339
Iteration:  30
[0 2 4 ... 8 6 0] [0 4 3 ... 8 0 0]
0.2785084745762712
Iteration:  40
[0 2 4 ... 8 0 0] [0 4 3 ... 8 0 0]
0.3287118644067797
Iteration:  50
[0 2 4 ... 8 0 0] [0 4 3 ... 8 0 0]
0.37342372881355934
Iteration:  60
[0 2 8 ... 8 0 0] [0 4 3 ... 8 0 0]
0.4096101694915254
Iteration:  70
[0 4 8 ... 8 0 0] [0 4 3 ... 8 0 0]
0.4433220338983051
Iteration:  80
[0 4 8 ... 3 0 0] [0 4 3 ... 8 0 0]
0.4795762711864407
Iteration:  90
[0 4 5 ... 3 0 0] [0 4 3 ... 8 0 0]
0.5111694915254237
Iteration:  100
[0 4 3 ... 3 0 0] [0 4 3 ... 8 0 0]
0.539271186440678
Iteration:  110
[0 4 3 ... 3 0 0] [0 4 3 ... 8 0 0]
0.5635084745762712
Iteration:  120
[0 4 3 ... 3 0 0] [0 4 3 ... 8 0 0]
0.5854406779661017
Iteration:  130
[0 4 3 ... 3 0 0] [0 4 3 ... 8 0 0]
0.6048983050847457
Iteration:  140
[